# Use model
I denne notebooken bruker jeg modellen som er lagd tidligere

In [6]:
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import matplotlib.pyplot as plt
import os
import plotly.express as px
import stockstats

from lib.rens import rens

In [7]:
def make_ready(filer):
    for i, fil in enumerate(filer):
        print(f"{i+1} av {len(filer)} renset" ,end='\r')
        data = cleanup.clean_data(pd.read_excel(fil))
        break  
    return data

def add_data(word):
    ordet = 'data/' + word
    return ordet

In [8]:
cleanup = rens()

In [9]:
filer = list(map(add_data, os.listdir('data/')))[61:] # 60 første er lagd til å bygge modell
fil = make_ready(filer)



,tid,open,close,high,low,volume,amount
0,9:0,196.00,196.00,196.00,196.00,203,203
1,9:2,198.75,198.75,198.75,198.75,310,310
2,9:3,199.20,199.20,199.20,199.20,330,330
3,9:5,199.90,199.90,199.90,199.90,113,113
4,9:6,198.90,198.90,198.90,198.90,1347,1347
...,...,...,...,...,...,...,...
350,16:16,198.35,198.35,198.35,198.35,304,304
351,16:17,198.30,198.30,198.30,198.30,55,55
352,16:18,198.35,198.35,198.35,198.35,1011,1011
353,16:19,198.95,198.95,198.95,198.95,1034,1034
